# Introduction to Wavelength Frame Multiplication

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn
import ess.wfm as wfm
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

## The long ESS pulse

Instruments at a pulsed neutron source,
assuming an idealized rectangular source pulse in time of length $t_{P}$,
is given by
$$\frac{\Delta \lambda}{\lambda} = \frac{t_{P}}{t} = \frac{t_{P}}{\alpha \lambda z_{\rm det}} ~, ~~~~~(1)$$
where $\lambda$ is the neutron wavelength, $t$ is time,
$\alpha = m_{\rm n}/h = 2.5278 \times 10^{-4}~{\rm s}\unicode{x212B}^{-1}{\rm m}^{-1}$
is the ratio of the neutron mass and the Planck constant,
and $z_{\rm det}$ is the distance from the source to the detector.

A natural consequence of this is that the wavelength resolution $\frac{\Delta \lambda}{\lambda}$
becomes finer with increasing wavelength.

This also means that the resolution can be pretty poor for a long-pulsed source such as the ESS.
A good way to visualize this is using a time-distance diagram,
which can represent the paths taken by the neutrons from the source to the detector.

In [ ]:
t_P = sc.scalar(2.860e+03, unit='us')
detector_position = sc.vector(value=[0., 0., 60.0], unit='m')
z_det = sc.norm(detector_position).value

fig0, ax0 = plt.subplots()
ax0.add_patch(
    Rectangle((0, 0),
              t_P.value,
              -0.05 * z_det,
              lw=1,
              fc='grey',
              ec='k',
              zorder=10))
# Indicate source pulse and add the duration.
ax0.text(0,
        -0.05 * z_det,
        "Source pulse ({} {})".format(t_P.value,
                                      t_P.unit),
        ha="left",
        va="top",
        fontsize=8)
ax0.plot([0, 1.0e4], [z_det] * 2, lw=3, color='grey')
ax0.text(0., z_det, 'Detector', ha='left', va='top')
# Draw 2 neutron paths
ax0.plot([0.02 * t_P.value, 4.0e3], [0, z_det], lw=2, color='r')
ax0.plot([t_P.value, 4.0e3], [0, z_det], lw=2, color='b')
ax0.text(3.7e3, 0.5*z_det, r'$\lambda_{1}$', ha='left', va='center', color='b')
ax0.text(1.5e3, 0.5*z_det, r'$\lambda_{2}$', ha='left', va='center', color='r')
ax0.set_xlabel("Time [microseconds]")
ax0.set_ylabel("Distance [m]")
ax0.set_title('Figure 1')
fig0.canvas

As illustrated in Fig. 1, with a long pulse, two neutrons with very different wavelengths
($\lambda_{1} < \lambda_{2}$) can reach the detector at the exact same time,
if they originated from a different part of the pulse.

This has the effect of merging many different wavelengths together on the same detector pixel,
making any conversion, from an event arrival time at the detector to a wavelength, difficult.

## So what is WFM anyway?

Within the concept of wavelength frame multiplication (WFM),
each source pulse is chopped into a number of sub pulses referred to as wavelength frames,
where each wavelength frame $N$ contains a subsequent part of the spectrum of the source pulse.

The main reason for using the WFM concept is to redefine the burst time $t_{P} = \Delta t$ as implied by Eq. (1), in order to match the required wavelength resolution of the experiment.

In [ ]:
t_0 = sc.scalar(5.0e+02, unit='us')
t_A = (t_0 + t_P).value
fig2, ax2 = plt.subplots()
ax2.add_patch(
    Rectangle((0, 0),
              (t_P + t_0).value,
              -0.05 * z_det,
              lw=1,
              fc='grey',
              ec='k',
              zorder=10))

# Indicate source pulse and add the duration.
ax2.text(0,
        -0.05 * z_det,
        "Source pulse",
        ha="left",
        va="top",
        fontsize=8)
ax2.plot([0, 3.1e4], [z_det] * 2, lw=3, color='grey')
ax2.text(0., z_det, 'Detector', ha='left', va='top')

dt = 1000.0
z_wfm = 15.0
xmin = 0.0
for i in range(3):
    xmax = 3000.0 + (i * 2.0 * dt)
    ax2.plot([xmin, xmax], [z_wfm] * 2, color='k')
    xmin = xmax + dt
ax2.plot([xmin, 3.1e4], [z_wfm] * 2, color='k')
ax2.text(25000.0, z_wfm, "WFM", ha='left', va='top')
ax2.plot([t_A, 5000.0], [0, z_det], color='r')
ax2.plot([2600.0, 5000.0], [0, z_det], color='b')
ax2.plot([t_A, 13000.0], [0, z_det], color='r')
ax2.plot([2400.0, 13000.0], [0, z_det], color='b')
ax2.plot([t_A, 21500.0], [0, z_det], color='r')
ax2.plot([2200.0, 21500.0], [0, z_det], color='b')

ax2.set_xlabel("Time [microseconds]")
ax2.set_ylabel("Distance [m]")
ax2.set_title('Figure 2')
fig2.canvas

## A closer look

### The ESS pulse shape

At a real beamline, the pulse shape is not rectangular,
but has rising and falling edges, as shown in Fig. 3.

In [ ]:
x = np.linspace(0, 5.0, 100)
a = 4.0
b = 0.0
c = 1.5e10
d = 3.0
e = 3.0
y = 2.0 * c / (np.exp(-a * (x - b)) + 1.0) - c
n = 60
y2 = c * np.exp(-e * (x - d))
y[n:] = y2[n:]
fig1, ax1 = plt.subplots()
ax1.plot(x, y, lw=2, color='k')

i1 = 5
i2 = 65
ax1.fill([x[i1]] + x[i1:i2].tolist() + [x[i2-1]],
         [0] + y[i1:i2].tolist() + [0], alpha=0.3)

fs = 15

ax1.axvline(x[i1], color='k')
ax1.axvline(x[i2-1], color='k')
ax1.text(x[i1], 1.58e10, r' $t_{0}$', ha='left', va='top', fontsize=fs)
ax1.text(x[i2-1], 1.58e10, r' $t_{\rm A}$', ha='left', va='top', fontsize=fs)
ax1.plot([4.5]*2, [0, 0.3e10], color='k')
ax1.text(4.5, 0.3e10, r'$t_{\rm B}$', ha='center', va='bottom', fontsize=fs)
ax1.annotate(text='', xy=(x[i1], 0.7e10), xytext=(x[i2-1], 0.7e10),
             arrowprops=dict(arrowstyle='<->'))
ax1.text(0.5*(x[i1] + x[i2-1]), 0.7e10, 'utilised\n pulse length', ha='center', va='bottom', fontsize=fs)
ax1.text(0.5*(x[i1] + x[i2-1]), 0.7e10, r'$t_{\rm P}$', ha='center', va='top', fontsize=fs)

ax1.set_ylim(0., 1.6e10)
ax1.set_xlabel("Time [ms]")
ax1.set_ylabel(r"Flux density $[{\rm n/s/cm}^2]$")
ax1.set_title('Figure 3')
fig1.canvas

Here we define several important quantities:

- The pulse $t_0$ is defined as the point in time at which the pulse is bright enough for the purposes of the experiment.
- $t_{\rm A}$ is the time when the flux has fallen down to a level below the required brightness.
- $t_{\rm P} = t_{\rm A} - t_{0}$ is the portion of the pulse that is used for the measurements, the analog of the pulse length for the ideal rectangular pulse above.
- $t_{\rm B}$ marks the end of the pulse; i.e. the time when the flux is considered to be effectively zero.

### Using a single WFM chopper

The effective burst time $\Delta t$ is defined by a WFM chopper (WFMC),
as illustrated in Fig. 3,
for the two limiting wavelengths $\lambda_{\rm min}$ and $\lambda_{\rm max}$ of a single wavelength frame $N$.

The wavelength frame is re-limited in a predefined time window by at least one frame overlap chopper (FOC)
that inhibits the overlap of neutrons from various frames, as indicated by the dashed lines in Fig. 3.
Their wavelength is labeled with $\lambda_{\rm min}^{'}$ and $\lambda_{\rm max}^{'}$.
The FOC is also removes undesired neutrons with the wrong wavelength that arise from the rising and falling edges
at the beginning and at the end of the source pulse.
Although their intensity is small, neutrons with an undesired wavelength would lead to an increased background.

In [ ]:
coords = wfm.make_fake_beamline(
    nframes=1,
    chopper_positions={
        "WFMC1": sc.vector(value=[0.0, 0.0, 4.5], unit='m'),
        "WFMC2": sc.vector(value=[0.0, 0.0, 5.5], unit='m')
    })
coords['position'] = sc.vector(value=[0., 0., 15.], unit='m')
ds = sc.Dataset(coords=coords)
frames = wfm.get_frames(ds)

z_det = sc.norm(ds.coords["position"]).value
t_0 = ds.coords["source_pulse_t_0"].value
t_A = (ds.coords["source_pulse_length"] + ds.coords["source_pulse_t_0"]).value
t_B = (ds.coords["source_pulse_length"] + 2.0*ds.coords["source_pulse_t_0"]).value
z_foc = 12.0
tmax_glob = 1.4e4
height = 0.02

chopper_wfm1 = coords["choppers"].value["WFMC1"]
chopper_wfm2 = coords["choppers"].value["WFMC2"]

fig3, ax3 = plt.subplots()
ax3.add_patch(
    Rectangle((0, 0),
              t_B,
              -height * z_det,
              lw=1,
              fc='lightgrey',
              ec='k',
              zorder=10))
ax3.add_patch(
    Rectangle((ds.coords["source_pulse_t_0"].value, 0),
              ds.coords["source_pulse_length"].value,
              -height * z_det,
              lw=1,
              fc='grey',
              ec='k',
              zorder=10))

# Indicate source pulse and add the duration.
ax3.text(ds.coords["source_pulse_t_0"].value,
        -height * z_det,
        r"$t_{0}$",
        ha="center",
        va="top",
        fontsize=8)
ax3.text(t_A,
        -height * z_det,
        r"$t_{A}$",
        ha="center",
        va="top",
        fontsize=8)
ax3.text(t_B,
        -height * z_det,
        r"$t_{B}$",
        ha="center",
        va="top",
        fontsize=8)

z_wfm = sc.norm(0.5 * (chopper_wfm1.position + chopper_wfm2.position)).value
xmin = chopper_wfm1.time_open.values
xmax = chopper_wfm1.time_close.values
dt = xmax - xmin

ax3.plot([0, xmin], [z_wfm] * 2, color='k')
ax3.plot([xmax, tmax_glob], [z_wfm] * 2, color='k')
ax3.text(tmax_glob, z_wfm, "WFMC", ha='right', va='top')


slope_lambda_max = z_wfm / (xmin - t_0)
slope_lambda_min = z_wfm / (xmax - t_A)
int_lambda_max = z_wfm - slope_lambda_max * xmin
int_lambda_min = z_wfm - slope_lambda_min * xmax
x_lambda_max = (z_det - int_lambda_max) / slope_lambda_max
x_lambda_min = (z_det - int_lambda_min) / slope_lambda_min

ax3.plot([t_0, x_lambda_max, x_lambda_max + dt, t_0 + dt],
         [0.0, z_det, z_det, 0], color='C0')
ax3.plot([t_A, x_lambda_min, x_lambda_min - dt, t_A - dt],
         [0.0, z_det, z_det, 0], color='C2')

x_lambda_max_foc = (z_foc - int_lambda_max) / slope_lambda_max + dt
x_lambda_min_foc = (z_foc - int_lambda_min) / slope_lambda_min - dt
ax3.plot([0, x_lambda_min_foc], [z_foc] * 2, color='k')
ax3.plot([x_lambda_max_foc, tmax_glob], [z_foc] * 2, color='k')
ax3.text(0.0, z_foc, "FOC", ha='left', va='top')


slope_lambda_min_prime = z_wfm / (xmin - t_B)
slope_lambda_max_prime = z_wfm / xmax
int_lambda_max_prime = z_wfm - slope_lambda_max_prime * xmax
int_lambda_min_prime = z_wfm - slope_lambda_min_prime * xmin
x_lambda_max_prime = (z_foc - int_lambda_max_prime) / slope_lambda_max_prime
x_lambda_min_prime = (z_foc - int_lambda_min_prime) / slope_lambda_min_prime

ax3.plot([t_B, x_lambda_min_prime], [0.0, z_foc], color='k', ls='dashed', lw=1)
ax3.plot([0, x_lambda_max_prime], [0.0, z_foc], color='k', ls='dashed', lw=1)

ax3.text(x_lambda_min - dt, z_det, r'$\lambda_{\rm min}$', ha='right', va='top', color='C2')
ax3.text(x_lambda_max + dt, z_det, r'$\lambda_{\rm max}$', ha='left', va='top', color='C0')
ax3.text(x_lambda_min_prime, z_foc, r"$\lambda_{\rm min}^{'}$", ha='left', va='top', color='k')
ax3.text(x_lambda_max_prime, z_foc, r"$\lambda_{\rm max}^{'}$", ha='left', va='top', color='k')

ax3.plot([xmin] * 2, [z_wfm, z_det + 1.0], lw=1, color='k')
ax3.plot([x_lambda_min - dt] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax3.plot([x_lambda_min] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax3.plot([x_lambda_max + dt] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax3.plot([x_lambda_max] * 2, [z_det, z_det + 1.0], lw=1, color='k')

ax3.fill([t_0 + dt, t_A - dt, 3013.10],
         [0, 0, 3.2963], color='mediumpurple', alpha=0.3, zorder=-2)
ax3.fill([x_lambda_min, x_lambda_max, 4515.077],
         [z_det, z_det, 6.704], color='mediumpurple', alpha=0.3, zorder=-2)

ax3.annotate(text='', xy=(xmin, z_det + 0.7), xytext=(x_lambda_min - dt, z_det + 0.7),
             arrowprops=dict(arrowstyle='<->'))
ax3.text(0.5*(xmin + x_lambda_min - dt), z_det + 0.7, r'$t(\lambda_{\rm min})$',
         va='bottom', ha='center')
ax3.text(x_lambda_min - 0.5*dt, z_det + 0.7, r'$\Delta t$',
         va='bottom', ha='center')
ax3.text(x_lambda_max + 0.5*dt, z_det + 0.7, r'$\Delta t$',
         va='bottom', ha='center')

ax3.plot([0, tmax_glob], [z_det] * 2, lw=3, color='grey')
ax3.text(0., z_det, 'Detector', ha='left', va='top')

ax3.grid(True, color='lightgray', linestyle="dotted")
ax3.set_axisbelow(True)
ax3.set_xlabel("Time [microseconds]")
ax3.set_ylabel("Distance [m]")
ax3.set_title('Figure 4')

fig3.canvas

The relative spectral resolution $\Delta \lambda / \lambda$ at the detector position $z_{\rm det}$
is defined by the burst time $\Delta t$ and the time-of-flight $t(\lambda)$ of the neutrons (see eq. (1)).
Because $\Delta t$t is independent of the wavelength for the case of a single WFM chopper disc,
$\Delta \lambda / \lambda$ depends on $\lambda$,
on the distance $z_{\rm WFM}$ of the WFM chopper from the source,
and on the detector position $z_{\rm det}$ as depicted in Fig. 4.

The WFM chopper acts as a virtual source,
reducing the effective burst time $\Delta t$,
while at the same time also reducing the effective time-of-flight to the detector from
$t(\lambda) = \alpha \lambda z_{\rm det}$ to $t(\lambda) = \alpha \lambda (z_{\rm det} - z_{\rm WFM})$.

A resolution that depends on $\lambda$ is not suited for some applications (such as imaging, reflectometry, ...)
where a constant $\Delta \lambda / \lambda$ is much more desirable.

## A pair of optically blind choppers

A constant wavelength resolution $\Delta \lambda / \lambda$ can be enforced by using a pair
of optically blind WFM chopper discs,
positioned at the positions $z_{\rm WFM1}$ and $z_{\rm WFM2} = z_{\rm WFM1} + \Delta z_{\rm WFM}$,
as shown in Fig. 5.

In this context, 'optically blind' indicates that the choppers have the same opening angles,
but the phase of the second chopper is shifted such that the second chopper opens exactly at
the time when the first chopper closes.
Such a setup introduces a wavelength dependence in the effective burst time
$\Delta t(\lambda) = \alpha \lambda \Delta z_{\rm WFM}$.

The time-of-flight to the detector remains the same as for the single WFM disc setup
described above ($t(\lambda) = \alpha \lambda (z_{\rm det} - z_{\rm WFM})$)
with $z_{\rm WFM} = \frac{1}{2} (z_{\rm WFM1} + z_{\rm WFM2})$
now representing the center position of the WFM chopper pair from the source.

The resolution for an idealized instrument with infinitesimally small beam cross-section can again be
calculated using Eq. (1):

$$\frac{\Delta \lambda}{\lambda} = \frac{\Delta t(\lambda)}{t(\lambda)} = \frac{\Delta z_{\rm WFM}}{z_{\rm det} - z_{\rm WFM}} ~. ~~~~~(2)$$

Because the term on the right hand side of Eq. (2) is constant,
the resolution becomes independent of the wavelength for an optically blind WFM chopper system.

In [ ]:
coords = wfm.make_fake_beamline(
    nframes=1,
    chopper_positions={
        "WFMC1": sc.vector(value=[0.0, 0.0, 4.5], unit='m'),
        "WFMC2": sc.vector(value=[0.0, 0.0, 5.5], unit='m')
    })
coords['position'] = sc.vector(value=[0., 0., 15.], unit='m')
ds = sc.Dataset(coords=coords)
frames = wfm.get_frames(ds)
fig4 = wfm.plot.time_distance_diagram(ds)
ax4 = fig4.get_axes()[0]

chopper_wfm1 = coords["choppers"].value["WFMC1"]
chopper_wfm2 = coords["choppers"].value["WFMC2"]
z_wfm = sc.norm(0.5 * (chopper_wfm1.position + chopper_wfm2.position)).value
xmin = chopper_wfm1.time_open.values
xmax = chopper_wfm1.time_close.values
z_foc = 12.0

ax4.plot([xmax] * 2, [z_wfm, z_det + 1.0], lw=1, color='k')
ax4.plot([0, frames["time_max"].values],
         [z_wfm] * 2, lw=1, color='k', ls='dotted')
ax4.text(frames["time_max"].values, z_wfm, r'$z_{\rm WFM}$', ha='left', va='center')

ax4.plot([0, 5770.5], [z_foc] * 2, color='k')
ax4.plot([9578.9, frames["time_max"].values], [z_foc] * 2, color='k')
ax4.text(frames["time_max"].values, z_foc, 'FOC', ha='right', va='bottom')

ax4.plot([(frames["time_min"] + frames["delta_time_min"]).values] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax4.plot([frames["time_min"].values] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax4.plot([(frames["time_max"] - frames["delta_time_max"]).values] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax4.plot([frames["time_max"].values] * 2, [z_det, z_det + 1.0], lw=1, color='k')

xmid = (0.5*((frames["time_min"] + frames["time_min"] + frames["delta_time_min"]).data)).values
ax4.plot([xmid] * 2, [z_det, z_det + 0.5], lw=1, color='k')

ax4.annotate(text='', xy=(xmax, z_det + 0.4), xytext=(xmid, z_det + 0.4),
             arrowprops=dict(arrowstyle='<->'))
ax4.text(0.5*(xmax + frames["time_min"].values), z_det + 0.4, r'$t(\lambda_{N})$',
         va='bottom', ha='center')
ax4.text(xmid, z_det + 1.0, r'$\Delta t(\lambda_{N})$', va='bottom', ha='center', color='C2')
ax4.text((0.5*((frames["time_max"] + frames["time_max"] - frames["delta_time_max"]).data)).values,
         z_det + 1.0, r'$\Delta t(\lambda_{N+1})$',
         va='bottom', ha='center', color='C0')
ax4.plot([xmax, xmid], [z_wfm, z_det], lw=1, ls='dashed', color='k')
ax4.text(frames['time_min'].values, z_det, r'$\lambda_{N}$   ', ha='right', va='top', color='C2')
ax4.text(frames['time_max'].values, z_det, r'$\lambda_{N+1}$', ha='left', va='top', color='C0')

ax4.lines[4].set_color('C2')
ax4.patches[2].set_color('mediumpurple')
ax4.set_xlim(-400, 12500)
ax4.set_title('Figure 5')

fig4.canvas

In a real WFM chopper system,
more than one frame can be realized by equipping the chopper with additional windows such that the full
band width that fits in the time period between two subsequent source pulses at the detector position is used.
This is illustrated in Fig. 6.

The frame overlap chopper defines the bandwidth of the frame $N$ in order to suppress cross-talk
from neighboring wavelength frames,
as illustrated in Fig. 6 for two subsequent wavelength frames $N = 1$ and $N = 2$.
They are designed such that $\lambda_{N, {\rm max}} = \lambda_{N+1, {\rm min}}$ yield a continuous spectrum
at the detector.
The concept of wavelength frame multiplication implies that no data can be recorded in the
time window between two subsequent wavelength frames.

In [ ]:
coords = wfm.make_fake_beamline(
    nframes=2,
    chopper_positions={
        "WFMC1": sc.vector(value=[0.0, 0.0, 4.5], unit='m'),
        "WFMC2": sc.vector(value=[0.0, 0.0, 5.5], unit='m')
    })
coords['position'] = sc.vector(value=[0., 0., 15.], unit='m')
ds = sc.Dataset(coords=coords)
frames = wfm.get_frames(ds)
fig5 = wfm.plot.time_distance_diagram(ds)
ax5 = fig5.get_axes()[0]

chopper_wfm1 = coords["choppers"].value["WFMC1"]
chopper_wfm2 = coords["choppers"].value["WFMC2"]
z_wfm = sc.norm(0.5 * (chopper_wfm1.position + chopper_wfm2.position)).value
xmin = chopper_wfm1.time_open.values
xmax = chopper_wfm1.time_close.values
z_foc = 12.0

# ax4.plot([xmax] * 2, [z_wfm, z_det + 1.0], lw=1, color='k')
ax5.plot([0, frames["time_max"].values[-1]],
         [z_wfm] * 2, lw=1, color='k', ls='dotted')
ax5.text(frames["time_max"].values[-1], z_wfm, r'$z_{\rm WFM}$', ha='left', va='center')

ax5.plot([(frames["time_min"] + frames["delta_time_min"]).values] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax5.plot([frames["time_min"].values] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax5.plot([(frames["time_max"] - frames["delta_time_max"]).values] * 2, [z_det, z_det + 1.0], lw=1, color='k')
ax5.plot([frames["time_max"].values] * 2, [z_det, z_det + 1.0], lw=1, color='k')

# xmid = (0.5*((frames["time_min"] + frames["time_min"] + frames["delta_time_min"]).data)).values
# ax4.plot([xmid] * 2, [z_det, z_det + 0.5], lw=1, color='k')

# ax4.annotate(text='', xy=(xmax, z_det + 0.4), xytext=(xmid, z_det + 0.4),
#              arrowprops=dict(arrowstyle='<->'))
ax5.text(xmid, z_det + 1.0, r'$\lambda_{N=1}$', va='bottom', ha='center', color='C2')
ax5.text((0.5*((frames["time_max"] + frames["time_max"] - frames["delta_time_max"]).data)).values[0],
         z_det + 1.0, r'$\lambda_{2}$',
         va='bottom', ha='center', color='C0')
ax5.text((0.5*((frames["time_min"] + frames["time_min"] + frames["delta_time_min"]).data)).values[-1],
         z_det + 1.0, r'$\lambda_{2}$',
         va='bottom', ha='center', color='C0')
ax5.text((0.5*((frames["time_max"] + frames["time_max"] - frames["delta_time_max"]).data)).values[-1],
         z_det + 1.0, r'$\lambda_{3}$',
         va='bottom', ha='center', color='C1')

# ax4.plot([xmax, xmid], [z_wfm, z_det], lw=1, ls='dashed', color='k')
# ax4.text(frames['time_min'].values, z_det, r'$\lambda_{N}$   ', ha='right', va='top', color='C2')
# ax4.text(frames['time_max'].values, z_det, r'$\lambda_{N+1}$', ha='left', va='top', color='C0')

ax5.lines[6].set_color('C2')
ax5.lines[8].set_color('C0')
ax5.patches[2].set_color('mediumpurple')
ax5.patches[5].set_color('grey')
# ax4.set_xlim(-400, 12500)


fig5.canvas

An additional feature of the WFM concept is the possibility to adjust the wavelength resolution by simply
changing the distance $\Delta z_{\rm WFM}$ between the WFM chopper discs (e.g. by using a motorized linear stage).
According to Eq. (2),
reducing $\Delta z_{\rm WFM}$ leads to a finer wavelength resolution at the cost of
intensity by effectively reducing the time window in which the neutrons can pass.

## A short example

We now proceed to illustrate data processing at a WFM beamline in the form of a short example.

### Create a beamline

We first create a beamline with two WFM choppers and 6 wavelength frames,
using the `wfm.make_fake_beamline` helper utility.
The detector is placed 60 m from the source.

In [ ]:
coords = wfm.make_fake_beamline(nframes=6)
ds = sc.Dataset(coords=coords)
wfm.plot.time_distance_diagram(ds).canvas

The properties of the frames (boundaries in time and wavelength) are computed using the `wfm.get_frames` function

In [ ]:
frames = wfm.get_frames(ds)
frames

### Create some neutrons

We create 6 neutrons with known wavelengths, one for each frame.
We choose the values for the wavelengths based on the limits given in the `frames` information above.

In [ ]:
wavelengths = sc.array(dims=['wavelength'],
                       values=[1.5, 3.0, 4.5, 6.0, 7.0, 8.25],
                       unit='angstrom')

We assume that all 6 neutrons originated half-way through the source pulse,
and we can thus calculate the times at which they hit the detector.

In [ ]:
# Neutron mass to Planck constant ratio
alpha = 2.5278e-4 * (sc.Unit('s') / sc.Unit('angstrom') / sc.Unit('m'))
# Distance between the detector pixel and the source
dz = sc.norm(coords['position'] - coords['source_position'])
# Compute arrival times, in microseconds
arrival_times = sc.to_unit(
    alpha * dz * wavelengths,
    'us') + coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])
arrival_times

### Wrap the neutron counts and the beamline into a DataArray

We make a data array that contains the beamlineinformation and a histogram of the neutrons over the time dimension.

In [ ]:
tmin = sc.min(arrival_times)
tmax = sc.max(arrival_times)
dt = 0.1 * (tmax - tmin)
coords['time'] = sc.linspace(dim='time',
                             start=(tmin - dt).value,
                             stop=(tmax + dt).value,
                             num=2001,
                             unit=dt.unit)
counts, _ = np.histogram(arrival_times.values, bins=coords['time'].values)
da = sc.DataArray(coords=coords,
                  data=sc.array(dims=['time'], values=counts, unit='counts'))
da

In [ ]:
da.plot()

### Stitch the frames

By using the information on the `frames`,
we apply a time correction on the data in each frame by subtracting the time of the mid-point between
the two WFM choppers from the `time` coordinate.

In [ ]:
stitched = wfm.stitch(frames=frames, data=da, dim='time', bins=2001)
stitched.plot()

The resulting coordinate is now time-of-flight,
and we can use `scippneutron` to convert the time-of-flight to wavelength.

In [ ]:
wav = scn.convert(stitched, origin='tof', target='wavelength', scatter=False)
wav

In [ ]:
wav.plot()

Zooming in on the first spike, we notice that something is not quite right: the original wavelength was 1.5 &#8491; but on the figure below is closer to 1.46 &#8491;.

In [ ]:
wav["wavelength", (1.2*sc.units.angstrom):(1.7*sc.units.angstrom)].plot()

Now the WFM method guarantees a constant $\Delta\lambda / \lambda$,
and so it is not surprising to see the final reduced wavelength not exactly matching the original.
The question is: is the error within $\Delta\lambda / \lambda$?

In [ ]:
# Distance between WFM choppers
dz_wfm = sc.norm(ds.coords["choppers"].value["WFMC2"].position - ds.coords["choppers"].value["WFMC1"].position)
# Delta_lambda  / lambda
dlambda_over_lambda = dz_wfm / sc.norm(coords['position'] -
                                       frames['wfm_chopper_mid_point'].data)
(1.5*sc.units.angstrom) * dlambda_over_lambda

At 1.5 &#8491;, the resolution is 0.0127 &#8491;, which is smaller than the offset we observe above.
In fact, we can perform a quick check using Scipp's value-based slicing, to verify that the sum of the counts
in a region $\Delta\lambda$ wide around the original wavelength should be equal to 1:

In [ ]:
for i in range(len(wavelengths)):
    lam = wavelengths["wavelength", i]
    dlam = 0.5 * dlambda_over_lambda * lam
    print("Lambda:", lam, ", count in range:", sc.sum(wav['wavelength', lam - dlam:lam + dlam]).value)

This reveals that the reduced wavelengths for the first two neutrons do not agree within the required precision.
We go back to our time-distance diagram and look at the paths taken by our 6 neutrons (plotted in red).

In [ ]:
fig6 = wfm.plot.time_distance_diagram(da)
ax6 = fig6.get_axes()[0]

for i in range(len(wavelengths)):
    ax6.plot([(coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])).value,
              arrival_times['wavelength', i].value],
             [0.0, sc.norm(coords['position']).value], color='r')
fig6.canvas

Taking a closer look at the WFM choppers,
we observe that the numbers we picked for the first two neutrons actually lead to unphysical paths:
they do not make it through the chopper openings!

In [ ]:
ax6.set_xlim(-1.0e3, 2.0e4)
ax6.set_ylim(-1.5, 10.0)
fig6.canvas.draw_idle()
fig6.canvas

So we modify our values so that neutrons 1 and 2 make it through:

In [ ]:
wavelengths = sc.array(dims=['wavelength'],
                       values=[1.75, 3.2, 4.5, 6.0, 7.0, 8.25],
                       unit='angstrom')
arrival_times = sc.to_unit(
    alpha * dz * wavelengths,
    'us') + coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])

In [ ]:
fig7 = wfm.plot.time_distance_diagram(da)
ax7 = fig7.get_axes()[0]

for i in range(len(wavelengths)):
    ax7.plot([(coords['source_pulse_t_0'] + (0.5 * coords['source_pulse_length'])).value,
              arrival_times['wavelength', i].value],
             [0.0, sc.norm(coords['position']).value], color='r')
ax7.set_xlim(-1.0e3, 2.0e4)
ax7.set_ylim(-1.5, 10.0)
fig7.canvas

And repeat the stitching process:

In [ ]:
tmin = sc.min(arrival_times)
tmax = sc.max(arrival_times)
dt = 0.1 * (tmax - tmin)
coords['time'] = sc.linspace(dim='time',
                             start=(tmin - dt).value,
                             stop=(tmax + dt).value,
                             num=2001,
                             unit=dt.unit)
counts, _ = np.histogram(arrival_times.values, bins=coords['time'].values)
da = sc.DataArray(coords=coords,
                  data=sc.array(dims=['time'], values=counts, unit='counts'))
stitched = wfm.stitch(frames=frames, data=da, dim='time', bins=2001)
wav = scn.convert(stitched, origin='tof', target='wavelength', scatter=False)

In [ ]:
wav["wavelength", (1.55*sc.units.angstrom):(1.95*sc.units.angstrom)].plot()

This time, the peak is much closer to 1.75 &#8491;,
and we can make sure the sum within the $\Delta\lambda$ range is 1 for all 6 neutrons:

In [ ]:
for i in range(len(wavelengths)):
    lam = wavelengths["wavelength", i]
    dlam = 0.5 * dlambda_over_lambda * lam
    print("Lambda:", lam, ", count in range:", sc.sum(wav['wavelength', lam - dlam:lam + dlam]).value)